In [10]:
import random
import csv
import os
from faker import Faker

fake = Faker()

# ----------------------------------------------------------------------
# CONFIG
# ----------------------------------------------------------------------
NUM_COMPANIES = 500
NUM_SHAREHOLDERS = 250
NUM_AUDITORS = 10
NUM_INVOICES = 1500

OUTPUT_DIR = "output_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------------------------------------------------
# NODE CREATION
# ----------------------------------------------------------------------
companies = [
    {
        "company_id": f"C{i}",
        "name": fake.company(),
        "country": random.choice(["US", "IN", "DE", "SG", "UK", "JP", "FR", "AU"]),
        "sector": random.choice(["FIN", "MFG", "TECH", "RETAIL", "HEALTH", "ENERGY", "FOOD"]),
        "risk_score": 0.0,
        "opportunity_score": 0.0,
        "is_supplier": False,
        "_pattern_marker": None,
        "_invoice_count": 0,  # Track actual invoice activity
    }
    for i in range(NUM_COMPANIES)
]

shareholders = [
    {
        "shareholder_id": f"S{i}",
        "name": fake.name(),
        "type": random.choice(["INDIVIDUAL", "FUND", "INSTITUTION"]),
        "risk_score": 0.0,
        "opportunity_score": 0.0,
        "_pattern_marker": None,
    }
    for i in range(NUM_SHAREHOLDERS)
]

auditors = [
    {
        "auditor_id": f"A{i}",
        "name": fake.company() + " Audit",
        "risk_level": "HIGH" if i < 2 else random.choice(["LOW", "MEDIUM", "MEDIUM", "LOW"]),
    }
    for i in range(NUM_AUDITORS)
]

invoices = [
    {
        "invoice_id": f"INV{i}",
        "amount": round(random.uniform(1000, 100000), 2),
        "status": random.choice(["PAID", "PENDING", "PAID", "PAID"]),  # Most paid
        "is_simulated": True,
        "_reserved": False,
    }
    for i in range(NUM_INVOICES)
]

# ----------------------------------------------------------------------
# EDGE ACCUMULATOR
# ----------------------------------------------------------------------
edges = []

# Track invoice assignments per company for realistic distribution
company_invoice_map = {c["company_id"]: [] for c in companies}

# ----------------------------------------------------------------------
# HELPER: Realistic Business Hierarchies
# ----------------------------------------------------------------------
def create_legitimate_corporate_groups(companies, num_groups=8):
    """
    Create 8-10 legitimate multi-level corporate groups with subsidiaries.
    These are GOOD companies with normal business structures.
    """
    out = []
    available = [c for c in companies if c["_pattern_marker"] is None]
    
    for g in range(num_groups):
        if len(available) < 6:
            break
            
        # Pick parent company
        parent = random.choice(available)
        available.remove(parent)
        parent["_pattern_marker"] = f"LEGIT_GROUP_{g}_PARENT"
        
        # Create 3-5 direct subsidiaries
        num_subs = random.randint(3, 5)
        subs = random.sample(available, min(num_subs, len(available)))
        
        for idx, sub in enumerate(subs):
            available.remove(sub)
            sub["_pattern_marker"] = f"LEGIT_GROUP_{g}_SUB_{idx}"
            out.append(("SUBSIDIARY_OF", sub["company_id"], parent["company_id"],
                       {"since_year": random.randint(2005, 2020)}))
            
            # Some subsidiaries have their own sub-subsidiaries
            if random.random() < 0.4 and len(available) > 0:
                sub_sub = random.choice(available)
                available.remove(sub_sub)
                sub_sub["_pattern_marker"] = f"LEGIT_GROUP_{g}_SUBSUB_{idx}"
                out.append(("SUBSIDIARY_OF", sub_sub["company_id"], sub["company_id"],
                           {"since_year": random.randint(2010, 2022)}))
    
    print(f"✓ Created {num_groups} legitimate corporate groups")
    return out

# ----------------------------------------------------------------------
# PATTERN 1 - SHELL COMPANY CHAIN (2-3 instances for detection)
# ----------------------------------------------------------------------
def embed_shell_pattern(companies, auditors, invoices, pattern_num=1):
    """
    Creates a suspicious shell company structure with LOW activity
    """
    high_risk = [a for a in auditors if a["risk_level"] == "HIGH"]
    if not high_risk:
        return []
    
    auditor = random.choice(high_risk)
    
    available = [c for c in companies if c["_pattern_marker"] is None]
    if len(available) < 5:
        return []
    
    main = random.choice(available)
    main["_pattern_marker"] = f"PATTERN1_{pattern_num}_MAIN"
    
    remaining = [c for c in available if c != main]
    chain = random.sample(remaining, 4)
    for idx, c in enumerate(chain):
        c["_pattern_marker"] = f"PATTERN1_{pattern_num}_CHAIN_{idx}"

    pattern_edges = []
    
    # Build deep subsidiary chain
    parent = main
    for c in chain:
        pattern_edges.append(
            ("SUBSIDIARY_OF", c["company_id"], parent["company_id"], 
             {"since_year": random.randint(2019, 2022)})  # Recent formations
        )
        parent = c

    # All audited by SAME high-risk auditor
    for c in [main] + chain:
        pattern_edges.append(("AUDITED_BY", c["company_id"], auditor["auditor_id"], {}))

    # CRITICAL: Only 1 invoice per company (VERY low activity)
    available_invs = [inv for inv in invoices if not inv["_reserved"]]
    if len(available_invs) < 5:
        return pattern_edges
    
    shell_invoices = random.sample(available_invs, 5)
    for idx, c in enumerate([main] + chain):
        inv = shell_invoices[idx]
        inv["_reserved"] = True
        c["_invoice_count"] = 1
        pattern_edges.append(("ISSUES_TO", c["company_id"], inv["invoice_id"], {}))

    print(f"  🚨 Pattern 1.{pattern_num} (Shell Company): Main={main['company_id']}, "
          f"Chain Length=4, Auditor={auditor['auditor_id']} (HIGH-RISK)")
    
    return pattern_edges

# ----------------------------------------------------------------------
# PATTERN 2 - CIRCULAR TRADE (2-3 instances)
# ----------------------------------------------------------------------
def embed_circular_trade(companies, pattern_num=1):
    """
    Creates closed-loop circular trading pattern
    """
    available = [c for c in companies if c["_pattern_marker"] is None]
    if len(available) < 4:
        return []
    
    cycle_size = random.choice([3, 4, 5])  # Variable cycle size
    cycle_nodes = random.sample(available, min(cycle_size, len(available)))
    
    for idx, c in enumerate(cycle_nodes):
        c["_pattern_marker"] = f"PATTERN2_{pattern_num}_CYCLE_{idx}"

    pattern_edges = []
    
    # Create perfect circular loop
    for i in range(len(cycle_nodes)):
        a = cycle_nodes[i]["company_id"]
        b = cycle_nodes[(i + 1) % len(cycle_nodes)]["company_id"]
        # Suspicious: high volumes in closed loop
        pattern_edges.append(
            ("SUPPLIES", a, b, {"annual_volume": random.randint(100, 200)})
        )

    print(f"  ⚠️  Pattern 2.{pattern_num} (Circular Trade): "
          f"Cycle={[c['company_id'] for c in cycle_nodes]}, Size={cycle_size}")
    
    return pattern_edges

# ----------------------------------------------------------------------
# PATTERN 3 - HIDDEN INFLUENCE (2-3 instances)
# ----------------------------------------------------------------------
def embed_hidden_influence(companies, shareholders, invoices, pattern_num=1):
    """
    Creates hidden influence through concentrated supply relationships
    """
    available_companies = [c for c in companies if c["_pattern_marker"] is None]
    available_shareholders = [s for s in shareholders if s["_pattern_marker"] is None]
    
    if len(available_companies) < 2 or len(available_shareholders) < 1:
        return [], None
    
    target = random.choice(available_companies)
    target["_pattern_marker"] = f"PATTERN3_{pattern_num}_TARGET"
    
    remaining_companies = [c for c in available_companies if c != target]
    supplier = random.choice(remaining_companies)
    supplier["_pattern_marker"] = f"PATTERN3_{pattern_num}_SUPPLIER"
    supplier["is_supplier"] = True
    
    influencer = random.choice(available_shareholders)
    influencer["_pattern_marker"] = f"PATTERN3_{pattern_num}_INFLUENCER"

    pattern_edges = []
    
    # High ownership stake (>25%)
    ownership_pct = random.randint(30, 50)
    pattern_edges.append(
        ("OWNS_SHARE", influencer["shareholder_id"], supplier["company_id"],
         {"percentage": ownership_pct})
    )
    
    # Supplier provides 100 invoices to target (will be >80% of target's total)
    available_invs = [inv for inv in invoices if not inv["_reserved"]]
    if len(available_invs) < 120:
        return pattern_edges, None
    
    supplier_invoices = random.sample(available_invs, 100)
    for inv in supplier_invoices:
        inv["_reserved"] = True
        pattern_edges.append(("ISSUES_TO", supplier["company_id"], inv["invoice_id"], {}))
        pattern_edges.append(("PAYS", target["company_id"], inv["invoice_id"], {}))
    
    supplier["_invoice_count"] = 100
    target["_invoice_count"] = 100
    
    # SUPPLIES relationship
    pattern_edges.append(
        ("SUPPLIES", supplier["company_id"], target["company_id"],
         {"annual_volume": 100})
    )
    
    # Store for later background generation
    target["_p3_target"] = True
    
    print(f"  ✨ Pattern 3.{pattern_num} (Hidden Influence): Influencer={influencer['shareholder_id']}, "
          f"Owns {ownership_pct}% of Supplier={supplier['company_id']}, Target={target['company_id']}")
    
    return pattern_edges, target

# ----------------------------------------------------------------------
# REALISTIC BACKGROUND: Legitimate Supply Chains
# ----------------------------------------------------------------------
def create_realistic_supply_chains(companies, num_chains=15):
    """
    Create 15-20 legitimate supply chains (Supplier → Manufacturer → Distributor → Retailer)
    These represent NORMAL business relationships with healthy transaction volumes
    """
    out = []
    available = [c for c in companies if c["_pattern_marker"] is None]
    
    for chain_id in range(num_chains):
        if len(available) < 3:
            break
        
        # Build 3-4 tier supply chain
        chain_length = random.randint(3, 4)
        chain_companies = random.sample(available, min(chain_length, len(available)))
        
        for comp in chain_companies:
            available.remove(comp)
            comp["_pattern_marker"] = f"SUPPLY_CHAIN_{chain_id}"
        
        # Create linear supply flow
        for i in range(len(chain_companies) - 1):
            supplier = chain_companies[i]
            buyer = chain_companies[i + 1]
            supplier["is_supplier"] = True
            
            # Normal, healthy volumes
            out.append(("SUPPLIES", supplier["company_id"], buyer["company_id"],
                       {"annual_volume": random.randint(30, 80)}))
    
    print(f"✓ Created {num_chains} legitimate supply chains")
    return out

# ----------------------------------------------------------------------
# BACKGROUND GENERATORS WITH REALISTIC DISTRIBUTIONS
# ----------------------------------------------------------------------
def generate_random_subsidiaries(companies, count=150):
    """Additional random subsidiary relationships (not in corporate groups)"""
    out = []
    eligible = [c for c in companies if c["_pattern_marker"] is None or 
                "LEGIT_GROUP" in c.get("_pattern_marker", "")]
    
    for _ in range(min(count, len(eligible) // 2)):
        if len(eligible) < 2:
            break
        ch, p = random.sample(eligible, 2)
        out.append(("SUBSIDIARY_OF", ch["company_id"], p["company_id"],
                    {"since_year": random.randint(1990, 2023)}))
    return out

def generate_realistic_ownership(companies, shareholders):
    """
    Realistic ownership distribution:
    - Most companies have 2-5 shareholders
    - Typical stakes: 5-30%, rarely above 50%
    - Institutional investors hold larger stakes
    """
    out = []
    for c in companies:
        # Skip pattern 3 suppliers (already have influencer)
        if c.get("_pattern_marker") and "PATTERN3" in c["_pattern_marker"] and "SUPPLIER" in c["_pattern_marker"]:
            # Add minor shareholders
            num_owners = random.randint(1, 3)
            available_sh = [s for s in shareholders if not s.get("_pattern_marker") or 
                           "PATTERN3" not in s.get("_pattern_marker", "")]
            if len(available_sh) < num_owners:
                continue
            owners = random.sample(available_sh, num_owners)
            remaining = 50  # Influencer has 30-50%, rest split among others
        else:
            num_owners = random.randint(2, 5)
            owners = random.sample(shareholders, min(num_owners, len(shareholders)))
            remaining = 100
        
        N = len(owners)
        for i, o in enumerate(owners):
            if i == N - 1:
                pct = max(5, remaining)
            else:
                # Weighted by shareholder type
                if o["type"] == "INSTITUTION":
                    max_stake = min(remaining - 5 * (N - i - 1), 35)
                    pct = random.randint(10, max(10, max_stake))
                elif o["type"] == "FUND":
                    max_stake = min(remaining - 5 * (N - i - 1), 25)
                    pct = random.randint(8, max(8, max_stake))
                else:  # INDIVIDUAL
                    max_stake = min(remaining - 5 * (N - i - 1), 20)
                    pct = random.randint(5, max(5, max_stake))
            
            remaining -= pct
            out.append(("OWNS_SHARE", o["shareholder_id"], c["company_id"], {"percentage": pct}))
    
    return out

def generate_realistic_audits(companies, auditors):
    """
    Realistic audit assignments:
    - Most companies audited by LOW/MEDIUM risk auditors
    - Pattern 1 companies already have HIGH-risk auditor
    """
    out = []
    for c in companies:
        if c.get("_pattern_marker") and "PATTERN1" in c["_pattern_marker"]:
            continue  # Already audited
        
        # Weighted selection: prefer LOW/MEDIUM risk auditors
        low_medium = [a for a in auditors if a["risk_level"] in ["LOW", "MEDIUM"]]
        if low_medium and random.random() < 0.9:
            auditor = random.choice(low_medium)
        else:
            auditor = random.choice(auditors)
        
        out.append(("AUDITED_BY", c["company_id"], auditor["auditor_id"], {}))
    return out

def generate_random_supplies(companies, count=300):
    """
    Additional random supply relationships (noise + legitimate business)
    Avoids pattern 2 companies to keep circular patterns clean
    """
    out = []
    eligible = [c for c in companies if not c.get("_pattern_marker") or 
                "PATTERN2" not in c.get("_pattern_marker", "")]
    
    for _ in range(count):
        if len(eligible) < 2:
            break
        a, b = random.sample(eligible, 2)
        
        # Most relationships have moderate volumes
        out.append(("SUPPLIES", a["company_id"], b["company_id"], 
                   {"annual_volume": random.randint(15, 70)}))
    return out

def generate_realistic_invoices(companies, invoices):
    """
    Realistic invoice distribution:
    - Active companies: 10-30 invoices
    - Moderate companies: 5-15 invoices
    - Inactive companies: 1-4 invoices
    - Pattern 3 targets get only 20 additional invoices (to maintain >80% concentration)
    """
    out = []
    available_invs = [inv for inv in invoices if not inv["_reserved"]]
    
    # Separate pattern 3 targets
    p3_targets = [c for c in companies if c.get("_p3_target")]
    other_companies = [c for c in companies if not c.get("_p3_target")]
    
    # Handle pattern 3 targets: only 20 additional invoices
    for target in p3_targets:
        if len(available_invs) < 20:
            break
        target_invs = random.sample(available_invs, 20)
        for inv in target_invs:
            available_invs.remove(inv)
            issuer = random.choice(other_companies)
            out.append(("ISSUES_TO", issuer["company_id"], inv["invoice_id"], {}))
            out.append(("PAYS", target["company_id"], inv["invoice_id"], {}))
            target["_invoice_count"] += 1
    
    # Distribute remaining invoices realistically
    # Create activity tiers
    for comp in other_companies:
        if comp.get("_invoice_count", 0) > 0:
            continue  # Already has invoices from patterns
        
        if not available_invs:
            break
        
        # Determine activity level
        if comp.get("_pattern_marker") and "PATTERN1" in comp.get("_pattern_marker", ""):
            continue  # Shell companies already have 1 invoice
        
        # Activity distribution: 60% active, 30% moderate, 10% low
        rand = random.random()
        if rand < 0.6:  # Active company
            num_invoices = random.randint(10, 25)
        elif rand < 0.9:  # Moderate company
            num_invoices = random.randint(5, 12)
        else:  # Low activity
            num_invoices = random.randint(2, 5)
        
        num_invoices = min(num_invoices, len(available_invs))
        comp_invs = random.sample(available_invs, num_invoices)
        
        for inv in comp_invs:
            available_invs.remove(inv)
            # Company issues some and pays some
            if random.random() < 0.5:
                out.append(("ISSUES_TO", comp["company_id"], inv["invoice_id"], {}))
                payer = random.choice([c for c in companies if c != comp])
                out.append(("PAYS", payer["company_id"], inv["invoice_id"], {}))
            else:
                issuer = random.choice([c for c in companies if c != comp])
                out.append(("ISSUES_TO", issuer["company_id"], inv["invoice_id"], {}))
                out.append(("PAYS", comp["company_id"], inv["invoice_id"], {}))
            
            comp["_invoice_count"] += 1
    
    return out

# ----------------------------------------------------------------------
# BUILD FINAL EDGE LIST
# ----------------------------------------------------------------------
print("=" * 80)
print("🏗️  GENERATING REALISTIC BUSINESS NETWORK WITH EMBEDDED FRAUD PATTERNS")
print("=" * 80)

print("\n📊 PHASE 1: Creating Legitimate Business Structures")
print("-" * 80)
edges.extend(create_legitimate_corporate_groups(companies, num_groups=10))
edges.extend(create_realistic_supply_chains(companies, num_chains=20))

print("\n🚨 PHASE 2: Embedding Fraud/Opportunity Patterns")
print("-" * 80)

# Embed 2-3 instances of each pattern for robust detection
for i in range(2):
    edges.extend(embed_shell_pattern(companies, auditors, invoices, pattern_num=i+1))

for i in range(3):
    edges.extend(embed_circular_trade(companies, pattern_num=i+1))

p3_targets = []
for i in range(2):
    p3_edges, p3_target = embed_hidden_influence(companies, shareholders, invoices, pattern_num=i+1)
    edges.extend(p3_edges)
    if p3_target:
        p3_targets.append(p3_target)

print("\n📈 PHASE 3: Generating Background Noise & Additional Relationships")
print("-" * 80)
edges.extend(generate_random_subsidiaries(companies, count=150))
print("✓ Added random subsidiary relationships")

edges.extend(generate_realistic_ownership(companies, shareholders))
print("✓ Generated ownership structures")

edges.extend(generate_realistic_audits(companies, auditors))
print("✓ Assigned auditors")

edges.extend(generate_random_supplies(companies, count=300))
print("✓ Created additional supply relationships")

edges.extend(generate_realistic_invoices(companies, invoices))
print("✓ Distributed invoices across network")

print(f"\n✅ Total edges generated: {len(edges)}")
print("=" * 80)

# ----------------------------------------------------------------------
# STATISTICS
# ----------------------------------------------------------------------
print("\n📊 NETWORK STATISTICS")
print("-" * 80)

shell_companies = [c for c in companies if c.get("_pattern_marker") and "PATTERN1" in c["_pattern_marker"]]
circular_companies = [c for c in companies if c.get("_pattern_marker") and "PATTERN2" in c["_pattern_marker"]]
hidden_inf_entities = [c for c in companies if c.get("_pattern_marker") and "PATTERN3" in c["_pattern_marker"]]
legit_companies = [c for c in companies if c.get("_pattern_marker") and "LEGIT_GROUP" in c["_pattern_marker"]]

print(f"Total Companies: {NUM_COMPANIES}")
print(f"  • Legitimate corporate groups: {len(legit_companies)} companies")
print(f"  • Shell company chains: {len(shell_companies)} companies (2 patterns)")
print(f"  • Circular trade participants: {len(circular_companies)} companies (3 patterns)")
print(f"  • Hidden influence pattern: {len(hidden_inf_entities)} companies (2 patterns)")
print(f"  • Independent/Noise: {NUM_COMPANIES - len(shell_companies) - len(circular_companies) - len(hidden_inf_entities) - len(legit_companies)} companies")
print(f"\nInvoice Activity Distribution:")
low_activity = len([c for c in companies if c["_invoice_count"] <= 3])
medium_activity = len([c for c in companies if 4 <= c["_invoice_count"] <= 10])
high_activity = len([c for c in companies if c["_invoice_count"] > 10])
print(f"  • Low activity (≤3 invoices): {low_activity} companies")
print(f"  • Medium activity (4-10): {medium_activity} companies")
print(f"  • High activity (>10): {high_activity} companies")
print("=" * 80)

# ----------------------------------------------------------------------
# WRITE NODE CSVS
# ----------------------------------------------------------------------
def write_csv(filename, rows, header):
    with open(os.path.join(OUTPUT_DIR, filename), "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=header, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(rows)

write_csv("companies.csv", companies,
          ["company_id", "name", "country", "sector", "risk_score", "opportunity_score", "is_supplier"])

write_csv("shareholders.csv", shareholders,
          ["shareholder_id", "name", "type", "risk_score", "opportunity_score"])

write_csv("auditors.csv", auditors,
          ["auditor_id", "name", "risk_level"])

write_csv("invoices.csv", invoices,
          ["invoice_id", "amount", "status", "is_simulated"])

# ----------------------------------------------------------------------
# WRITE EDGE CSVS
# ----------------------------------------------------------------------
def edge_to_dict(rel_type, start, end, props):
    if rel_type == "SUBSIDIARY_OF":
        d = {
            "child_company_id": start, 
            "parent_company_id": end, 
            "since_year": props.get("since_year", 2020)
        }
        return d
    elif rel_type == "OWNS_SHARE":
        return {
            "shareholder_id": start, 
            "company_id": end, 
            "percentage": props.get("percentage", 10)
        }
    elif rel_type == "SUPPLIES":
        return {
            "supplier_company_id": start, 
            "buyer_company_id": end, 
            "annual_volume": props.get("annual_volume", 50)
        }
    elif rel_type == "AUDITED_BY":
        return {"company_id": start, "auditor_id": end}
    elif rel_type == "ISSUES_TO":
        return {"company_id": start, "invoice_id": end}
    elif rel_type == "PAYS":
        return {"company_id": start, "invoice_id": end}
    return {"start": start, "end": end}

def write_edges_by_type(edges):
    buckets = {
        "SUBSIDIARY_OF": [],
        "OWNS_SHARE": [],
        "SUPPLIES": [],
        "AUDITED_BY": [],
        "ISSUES_TO": [],
        "PAYS": [],
    }

    for rel, start, end, props in edges:
        buckets[rel].append(edge_to_dict(rel, start, end, props))

    for rel, rows in buckets.items():
        if rel == "SUBSIDIARY_OF":
            header = ["child_company_id", "parent_company_id", "since_year"]
            filename = "subsidiary_of.csv"
        elif rel == "OWNS_SHARE":
            header = ["shareholder_id", "company_id", "percentage"]
            filename = "owns_share.csv"
        elif rel == "SUPPLIES":
            header = ["supplier_company_id", "buyer_company_id", "annual_volume"]
            filename = "supplies.csv"
        elif rel == "AUDITED_BY":
            header = ["company_id", "auditor_id"]
            filename = "audited_by.csv"
        elif rel == "ISSUES_TO":
            header = ["company_id", "invoice_id"]
            filename = "issues_to.csv"
        elif rel == "PAYS":
            header = ["company_id", "invoice_id"]
            filename = "pays.csv"

        with open(os.path.join(OUTPUT_DIR, filename), "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()
            writer.writerows(rows)
        
        print(f"✓ {filename}: {len(rows)} relationships")

write_edges_by_type(edges)

print("\n" + "=" * 80)
print(f"✅ All CSV files successfully generated in: {OUTPUT_DIR}/")
print("=" * 80)
print("\n🎯 PATTERN DETECTION GUIDE:")
print("  1. Shell Companies: Look for chains with HIGH-RISK auditor + low invoice count (≤1)")
print("  2. Circular Trade: Detect cycles in SUPPLIES relationships with high volumes (>100)")
print("  3. Hidden Influence: Find shareholders with >25% ownership in suppliers providing >80% of invoices")
print("=" * 80)

🏗️  GENERATING REALISTIC BUSINESS NETWORK WITH EMBEDDED FRAUD PATTERNS

📊 PHASE 1: Creating Legitimate Business Structures
--------------------------------------------------------------------------------
✓ Created 10 legitimate corporate groups
✓ Created 20 legitimate supply chains

🚨 PHASE 2: Embedding Fraud/Opportunity Patterns
--------------------------------------------------------------------------------
  🚨 Pattern 1.1 (Shell Company): Main=C60, Chain Length=4, Auditor=A0 (HIGH-RISK)
  🚨 Pattern 1.2 (Shell Company): Main=C123, Chain Length=4, Auditor=A1 (HIGH-RISK)
  ⚠️  Pattern 2.1 (Circular Trade): Cycle=['C297', 'C282', 'C50', 'C409', 'C2'], Size=5
  ⚠️  Pattern 2.2 (Circular Trade): Cycle=['C450', 'C271', 'C207'], Size=3
  ⚠️  Pattern 2.3 (Circular Trade): Cycle=['C364', 'C233', 'C211'], Size=3
  ✨ Pattern 3.1 (Hidden Influence): Influencer=S221, Owns 38% of Supplier=C232, Target=C269
  ✨ Pattern 3.2 (Hidden Influence): Influencer=S83, Owns 36% of Supplier=C184, Target=C193

